In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# 1. 데이터 로딩
df = pd.read_excel('통합된_수출_예측_데이터셋.xlsx', index_col='날짜', parse_dates=True) # 데이터 통합본 넣기

# 2. 피처 & 타깃 분리
X = df.drop(columns=['수출량'])  # 수출량 제외 모든 피처
y = df['수출량']  # 타겟

# 3. 시계열 순서 유지한 학습/테스트 분리
split_idx = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# 4. DMatrix 변환
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

# 5. 파라미터 설정 및 학습
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}
xgb_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=200,
    early_stopping_rounds=50,
    evals=[(dtrain, 'train'), (dtest, 'eval')]
)

# 6. 예측
preds = xgb_model.predict(dtest)

# 7. 평가
def get_reg_eval(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

get_reg_eval(y_test, preds)
